# INST326 — Week 7 Exercises: Error Handling & Testing (Library Management)

**Focus (Week 7 only):** exception handling (`try` / `except` / `else` / `finally`), raising exceptions, defining simple custom exceptions, and **basic** unit testing with `unittest` (no fixtures beyond `setUp`/`tearDown`).

**Out of scope:** Anything introduced in Week 8 or later (e.g., inheritance, abstract classes, polymorphism, advanced testing/CI, design patterns).

> Context: Use a simple Library Management domain—books, members, catalog, and loans—to complete the tasks.


### Starter Helpers (Optional)

You may use or modify the minimal scaffolding below in your solutions. It intentionally avoids Week 8+ concepts.


In [45]:
# Minimal, Week-7-safe scaffolding (no inheritance/ABCs).
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Optional, Dict, List
import json

class LibraryError(Exception):
    """Base library-related error for Week 7."""

class DuplicateBookError(LibraryError):
    pass

class OverdueLoanError(LibraryError):
    pass

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

@dataclass
class Member:
    member_id: str
    email: str

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def check_overdue(self) -> bool:
        if datetime.now() > self.due_date and not self.returned:
            return True
        return False

class Catalog:
    def __init__(self):
        self._books: Dict[str, Book] = {}

    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN already exists: {book.isbn}")
        if book.copies < 0:
            raise ValueError("copies must be non-negative")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def load_from_json(self, path: str) -> int:
        # Intentionally minimal; implement robust handling in exercises.
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        count = 0
        for item in data:
            self.add_book(Book(isbn=item["isbn"], title=item["title"], copies=item.get("copies", 1)))
            count += 1
        return count


## 1) Validate ISBN with exceptions

Write a function `validate_isbn(isbn: str) -> str` that:
- Strips hyphens/spaces
- Verifies it is 10 or 13 digits (numeric only after stripping)
- Raises `ValueError` with a helpful message when invalid
Return the normalized ISBN string when valid.

In [46]:
# Your code here
def validate_isbn(isbn: str) -> str:
    if not isinstance(isbn, str):
        raise ValueError("ISBN must be a string")
    normalized = isbn.replace("-", "").replace(" ", "")
    if not (len(normalized) == 10 or len(normalized) == 13):
        raise ValueError(f"ISBN must be 10 or 13 digits after removing hyphens/spaces: got '{normalized}'")
    if not (normalized.isdigit()):
        # allow X/x only in last position for ISBN-10? The instructions requested numeric only after stripping,
        # so we enforce digits only.
        raise ValueError(f"ISBN must contain digits only after stripping: got '{normalized}'")
    return normalized

# Example:
print(validate_isbn("978-1-4028-9462-6"))  # -> '9781402894626'

9781402894626


## 2) Safe integer input for copies

Implement `parse_copies(text: str) -> int` that parses a positive integer for number of copies.
Use `try/except` to catch `ValueError`, and raise your own `ValueError` with a user-friendly message.

In [47]:
# Your code here
def parse_copies(text: str) -> int:
    try:
        val = int(text)
    except Exception:
        raise ValueError("copies must be a positive integer")
    if val < 0:
        raise ValueError("copies must be a positive integer")
    return val

# Examples:
#parse_copies("3") # 3
#parse_copies("three") # ValueError('copies must be a positive integer')

## 3) Custom exception: OverdueLoanError

Extend `Loan.check_overdue()` to **raise** `OverdueLoanError` if the book is overdue (instead of returning `True/False`). 
Catch this exception in a separate function `assert_not_overdue(loan: Loan)` that returns `True` when ok and `False` when overdue.

In [48]:
# Your code here
def assert_not_overdue(loan: Loan) -> bool:
    try:
        loan.check_overdue()  # will raise OverdueLoanError if overdue
        return True
    except OverdueLoanError:
        return False



## 4) Robust date parsing

Write `parse_date(text: str) -> datetime` that accepts `YYYY-MM-DD`. If parsing fails, raise `ValueError('invalid date: ...')`.
Use `try/except` around `datetime.strptime`.

In [49]:
# Your code here
from datetime import datetime

def parse_date(text: str) -> datetime:
    try:
        return datetime.strptime(text, "%Y-%m-%d")
    except Exception:
        raise ValueError(f"invalid date: {text}")

# Example:
parse_date("2025-10-27")

datetime.datetime(2025, 10, 27, 0, 0)

## 5) Late fee calculation with ZeroDivisionError guard

Define `per_day_late_fee(total_fee: float, days_late: int) -> float` that computes `total_fee / days_late`.
If `days_late` is 0, raise `ZeroDivisionError` with a clear message. Show a `try/except` usage example that prints a friendly message instead of crashing.

In [50]:
# Your code here
def per_day_late_fee(total_fee: float, days_late: int) -> float:
    if days_late == 0:
        raise ZeroDivisionError("days_late is zero — cannot divide by zero")
    return float(total_fee) / float(days_late)

# try/except demo here
# Demo try/except:
def demo_per_day_late_fee():
    try:
        print(per_day_late_fee(5.0, 0))
    except ZeroDivisionError as e:
        print("Friendly message:", e)

## 6) Always-close file with try/finally

Implement `read_file_head(path: str, n: int=3) -> list[str]` that opens a UTF-8 text file and returns the first `n` lines (stripped).
Use explicit `try/finally` to ensure closing (even though `with` is preferred) to practice `finally` behavior.

In [51]:
# Your code here
def read_file_head(path: str, n: int=3) -> list[str]:
    f = None
    lines: List[str] = []
    try:
        f = open(path, "r", encoding="utf-8")
        for i in range(n):
            line = f.readline()
            if line == "":
                break
            lines.append(line.rstrip("\n"))
    finally:
        if f:
            try:
                f.close()
            except Exception:
                pass
    return lines

## 7) Safe JSON catalog loading

Implement `safe_load_catalog(path: str, catalog: Catalog) -> int` that handles:
- `FileNotFoundError` → return 0
- `json.JSONDecodeError` → return 0
- `DuplicateBookError` → skip duplicate and continue
Return the number of **new** books added.

In [52]:
# Your code here
import json

def safe_load_catalog(path: str, catalog: Catalog) -> int:
    new_added = 0
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return 0

    for item in data:
        try:
            book = Book(isbn=item["isbn"], title=item.get("title", ""), copies=item.get("copies", 1))
            catalog.add_book(book)
            new_added += 1
        except DuplicateBookError:
            # skip duplicates and continue
            continue
        except Exception:
            # If other errors occur (e.g., invalid copies) skip the item
            continue
    return new_added

## 8) Duplicate book detection

Write `add_unique_book(catalog: Catalog, book: Book)` that raises `DuplicateBookError` if the ISBN is already present, else adds it.
Demonstrate `try/except` around this call to log a brief message and continue.

In [53]:
# Your code here
def add_unique_book(catalog: Catalog, book: Book) -> None:
    if book.isbn in catalog._books:
        raise DuplicateBookError(f"ISBN already exists: {book.isbn}")
    catalog.add_book(book)

# Demo with try/except here
def demo_add_unique(catalog: Catalog, book: Book):
    try:
        add_unique_book(catalog, book)
    except DuplicateBookError as e:
        print("Duplicate:", e)

## 9) Timeout handling (simulated)

Create `fetch_cover_image(isbn: str, timeout_s: float=0.1) -> bytes` that **simulates** a timeout by raising `TimeoutError` when `timeout_s` < 0.05.
Write `get_cover_or_none(isbn)` that calls it in `try/except TimeoutError` and returns `None` on timeout.

In [54]:
# Your code here
def fetch_cover_image(isbn: str, timeout_s: float=0.1) -> bytes:
    # Simulate network behavior: if timeout too small, raise TimeoutError
    if timeout_s < 0.05:
        raise TimeoutError("Simulated timeout")
    # simulate returning bytes
    return f"cover-for-{isbn}".encode("utf-8")

def get_cover_or_none(isbn: str):
    try:
        return fetch_cover_image(isbn, timeout_s=timeout_s)
    except TimeoutError:
        return None

## 10) Sanitizing member IDs

Implement `sanitize_member_id(value) -> str` that:
- Raises `TypeError` if not `str`
- Strips spaces, uppercases
- Raises `ValueError` if final form is empty or contains non-alphanumeric chars

In [55]:
# Your code here
def sanitize_member_id(value) -> str:
    if not isinstance(value, str):
        raise TypeError("member id must be a string")
    v = value.strip().upper()
    if v == "" or not v.isalnum():
        raise ValueError("member id must be non-empty and alphanumeric after cleaning")
    return v

## 11) Convert asserts to exceptions

Given legacy code that uses `assert copies >= 0`, replace it with explicit `if` + `raise ValueError('copies must be non-negative')` in a function `validate_copies(copies: int) -> int` that returns the validated value.

In [56]:
# Your code here
def validate_copies(copies: int) -> int:
    if copies < 0:
        raise ValueError("copies must be non-negative")
    return copies

## 12) Basic unit tests for `validate_isbn`

Create a `tests/`-style cell using `unittest` that verifies:
- Valid 10- and 13-digit ISBNs pass
- Bad inputs raise `ValueError`
Do **not** import any third-party packages.

In [57]:
# Your code here
import unittest

class TestValidateISBN(unittest.TestCase):
    def test_valid_10_and_13(self):
        self.assertEqual(validate_isbn("0123456789"), "0123456789")
        self.assertEqual(validate_isbn("9780132350884"), "9780132350884")
        self.assertEqual(validate_isbn("978-0-13-235088-4"), "9780132350884")
    def test_invalid_values(self):
        with self.assertRaises(ValueError):
            validate_isbn("abc")
        with self.assertRaises(ValueError):
            validate_isbn("123456789")   # 9 digits
        with self.assertRaises(ValueError):
            validate_isbn("123456789012")  # 12 digits

if __name__ == '__main__':
    unittest.main(argv=['-v'], exit=False)  # Uncomment to run in notebook

.....
----------------------------------------------------------------------
Ran 5 tests in 0.007s

OK


## 13) Unit test with `assertRaises`

Write a `unittest.TestCase` verifying that adding a duplicate ISBN to a `Catalog` raises `DuplicateBookError`.
Use `setUp` to create a fresh `Catalog` and seed it with one book.

In [58]:
# Your code here
import unittest

class TestCatalogDuplicates(unittest.TestCase):
    def setUp(self):
        self.catalog = Catalog()
        self.book = Book(isbn="1111111111", title="Test Book", copies=1)
        self.catalog.add_book(self.book)

    def test_duplicate_raises(self):
        with self.assertRaises(DuplicateBookError):
            self.catalog.add_book(Book(isbn="1111111111", title="Other", copies=1))

if __name__ == '__main__':
    unittest.main(argv=['-v'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.011s

OK


## 14) Subtests for multiple bad IDs (optional pattern)

Using `unittest`, write a single test method that loops over a collection of **invalid** member IDs and, using `self.subTest`, asserts that `sanitize_member_id` raises `ValueError` for each.

In [59]:
# Your code here
import unittest

class TestMemberIdSanitization(unittest.TestCase):
    def test_bad_ids(self):
        bad_values = ["", "   ", "abc!", "id with space", "###"]
        bad_values = ["", "   ", "abc!", "id with space", "###"]
        for v in bad_values:
            with self.subTest(v=v):
                with self.assertRaises((ValueError, TypeError)):
                    sanitize_member_id(v)

if __name__ == '__main__':
    unittest.main(argv=['-v'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK


## 15) try/except/else logging

Write `try_index_book(catalog: Catalog, book: Book) -> bool` that
- Tries to `add_book`
- On exception, returns `False`
- Uses `else` to return `True` only when no exception occurred

In [60]:
# Your code here
def try_index_book(catalog: Catalog, book: Book) -> bool:
    try:
        catalog.add_book(book)
    except Exception:
        return False
    else:
        return True

## 16) Graceful KeyboardInterrupt

Create `interactive_copies_prompt()` that repeatedly prompts the user for copies (use `input()`), converts using `parse_copies`, and prints the value.
If the user hits `Ctrl+C`, catch `KeyboardInterrupt` and print `Bye!` before returning.

In [61]:
# Your code here
def interactive_copies_prompt():
    try:
        while True:
            val = input("Enter copies (or blank to stop): ")
            if val == "":
                break
            try:
                copies = parse_copies(val)
            except ValueError as e:
                print("Bad input:", e)
                continue
            print("You entered:", copies)
    except KeyboardInterrupt:
        print("\nBye!")
        return


## 17) Catalog validator that accumulates errors

Implement `validate_catalog_items(items: list[dict]) -> list[str]` that walks a list of book dicts and **collects** error messages rather than raising immediately.
Return a list of error strings (empty if valid).

In [62]:
def validate_catalog_items(items: list[dict]) -> list[str]:
    errors: List[str] = []
    for idx, it in enumerate(items):
        try:
            if "isbn" not in it:
                raise ValueError("missing isbn")
            validate_isbn(it["isbn"])
            # copies optional
            if "copies" in it:
                parse_copies(it["copies"])
        except Exception as e:
            errors.append(f"item[{idx}]: {e}")
    return errors

## 18) Replace broad except

Refactor a function that currently uses a bare `except:` to instead catch **only** `ValueError` and `TypeError` and re-raise unknown exceptions. Provide a before/after example and explain why broad except is risky.

In [63]:
# Your code here
def before_style(x):
    try:
        return int(x)
    except:
        return 0  # BAD: swallows everything

def after_style(x):
    try:
        return int(x)
    except (ValueError, TypeError):
        return 0
    # other exceptions will bubble up


## 19) Exit codes on failure

Write `main(argv)` that attempts to load a catalog JSON path from `argv[1]` using `safe_load_catalog`.
- Return `0` on success, `2` on `FileNotFoundError`, `3` on `json.JSONDecodeError`, otherwise `1`.
Use `sys.exit(main(sys.argv))` pattern in a protected `if __name__ == '__main__':` block (comment it out in notebook).

In [64]:
# Your code here
import sys

def main(argv: list[str]) -> int:
    if len(argv) < 2:
        print("Usage: program <catalog.json>")
        return 1
    path = argv[1]
    catalog = Catalog()
    try:
        added = safe_load_catalog(path, catalog)
        print(f"Added {added} books")
        return 0
    except FileNotFoundError:
        return 2
    except json.JSONDecodeError:
        return 3
    except Exception:
        return 1


#if __name__ == '__main__':
    sys.exit(main(sys.argv))

    # in notebook, do not call sys.exit

## 20) End-to-end happy path test

Using `unittest`, write a test that:
- Builds a fresh `Catalog`
- Adds a valid `Book`
- Creates a `Loan` due tomorrow and asserts `assert_not_overdue` returns `True`
This is a **basic** end-to-end happy path—no Week 8+ features.

In [65]:
# Your code here
import unittest
from datetime import datetime, timedelta

class TestHappyPath(unittest.TestCase):
    def test_happy_flow(self):
        cat = Catalog()
        book = Book(isbn="9999999999", title="Happy Book", copies=1)
        cat.add_book(book)
        # Create loan due tomorrow
        due = datetime.now() + timedelta(days=1)
        loan = Loan(isbn=book.isbn, member_id="M1", due_date=due, returned=False)
        # assert_not_overdue should return True (not overdue)
        self.assertTrue(assert_not_overdue(loan))

if __name__ == '__main__':
    unittest.main(argv=['-v'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK


## Python skills you'll need (Weeks 1–7)

- **Core syntax & data types:** variables, strings, ints/floats, booleans
- **Collections:** lists, dicts (basic use only)
- **Control flow:** `if/elif/else`, `for` loops, `while` loops
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O:** open/read/write text and JSON (basic)
- **Classes & objects:** defining simple classes, `__init__`, instance methods, attributes
- **Encapsulation basics:** private attributes (naming convention), validation via methods/properties
- **Methods:** instance/class/static methods (as introduced in Week 6)
- **Error handling:** `try` / `except` / `else` / `finally`, `raise`, custom exceptions (Week 7)
- **Basic testing:** `unittest.TestCase`, `assertRaises`, `setUp`/`tearDown`, `subTest` (Week 7)
- **Standard library familiarity:** `datetime`, `json`, `sys`, built-in exceptions
